## Deliverable 2. Create a Customer Travel Destinations Map.

In [11]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [12]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Lebu,CL,-37.6167,-73.6500,54.73,69,27,9.46,scattered clouds
1,1,Lorengau,PG,-2.0226,147.2712,80.65,82,43,9.48,scattered clouds
2,2,Rikitea,PF,-23.1203,-134.9692,74.77,82,96,20.58,overcast clouds
3,3,Tuatapere,NZ,-46.1333,167.6833,50.86,96,100,1.54,overcast clouds
4,4,Kalino,RU,58.2504,57.6085,41.95,84,100,8.66,overcast clouds


In [15]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp=float(input("What is the minimum temperature you would like for your trip? "))
max_temp=float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [16]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
clean_df=city_data_df.loc[(city_data_df["Max Temp"] <=max_temp) & \
                                    (city_data_df["Max Temp"] >=min_temp)]
clean_df.head(5)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Lorengau,PG,-2.0226,147.2712,80.65,82,43,9.48,scattered clouds
5,5,Harper,LR,4.3750,-7.7169,77.68,88,79,7.72,light rain
7,7,Bengkulu,ID,-3.8004,102.2655,76.21,89,80,2.89,broken clouds
14,14,Akyab,MM,20.1500,92.9000,76.23,84,100,5.14,overcast clouds
19,19,Salalah,OM,17.0151,54.0924,77.54,68,23,3.47,few clouds


In [17]:
# 4a. Determine if there are any empty rows.
clean_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [18]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
#pandas dataframe drop na link https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dropna.html
clean_df.dropna()
clean_df.head(5)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Lorengau,PG,-2.0226,147.2712,80.65,82,43,9.48,scattered clouds
5,5,Harper,LR,4.3750,-7.7169,77.68,88,79,7.72,light rain
7,7,Bengkulu,ID,-3.8004,102.2655,76.21,89,80,2.89,broken clouds
14,14,Akyab,MM,20.1500,92.9000,76.23,84,100,5.14,overcast clouds
19,19,Salalah,OM,17.0151,54.0924,77.54,68,23,3.47,few clouds


In [19]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Lorengau,PG,80.65,scattered clouds,-2.0226,147.2712,
5,Harper,LR,77.68,light rain,4.3750,-7.7169,
7,Bengkulu,ID,76.21,broken clouds,-3.8004,102.2655,
14,Akyab,MM,76.23,overcast clouds,20.1500,92.9000,
19,Salalah,OM,77.54,few clouds,17.0151,54.0924,
24,Atuona,PF,78.15,few clouds,-9.8000,-139.0333,
32,Chinsali,ZM,79.63,few clouds,-10.5414,32.0816,
33,Hithadhoo,MV,82.24,heavy intensity rain,-0.6000,73.0833,
35,Sorong,ID,79.38,overcast clouds,-0.8833,131.2500,
46,Abu Dhabi,AE,86.11,clear sky,24.4667,54.3667,


In [36]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        hotel_df["Hotel Name"].replace("",np.nan,inplace=True)

In [37]:
hotel_df.isna().sum()

City                    0
Country                 0
Max Temp                0
Current Description     0
Lat                     0
Lng                     0
Hotel Name             18
dtype: int64

In [39]:
# 7. Drop the rows where there is no Hotel Name
clean_hotel_df=hotel_df.dropna(subset=["Hotel Name"])
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Lorengau,PG,80.65,scattered clouds,-2.0226,147.2712,Lorengau Harbourside Hotel
5,Harper,LR,77.68,light rain,4.3750,-7.7169,Screensaver
7,Bengkulu,ID,76.21,broken clouds,-3.8004,102.2655,Grage Hotel Bengkulu
14,Akyab,MM,76.23,overcast clouds,20.1500,92.9000,Hotel Memory
19,Salalah,OM,77.54,few clouds,17.0151,54.0924,"HAMDAN PLAZA HOTEL SALALAH, an HTG Hotel"
...,...,...,...,...,...,...,...
702,Diffa,NE,88.36,clear sky,13.3154,12.6113,ProDAF/Diffa
703,Atambua,ID,75.99,scattered clouds,-9.1061,124.8925,Pastori 1 Jemaat Polycarpus
707,Honiara,SB,76.39,broken clouds,-9.4333,159.9500,Heritage Park Hotel
713,Sola,VU,81.00,few clouds,-13.8833,167.5500,Leumerus Bungalows


In [41]:
# 8a. Create the output File (CSV)
output_data_file="WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [42]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [44]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))

In [ ]:
#Click the download icon in the upper left corner of the figure to same an image as a .png